In [2]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import plotly.express as px

# === FILE PATH ===
FILE_PATH = "student_data.xlsx"  # Change to "student_data.csv" if needed

# === LOAD DATA ===
if FILE_PATH.endswith(".xlsx"):
    df = pd.read_excel(FILE_PATH)
else:
    df = pd.read_csv(FILE_PATH)

# === DEFINE COLUMNS ===
class_cols = [f"Class {i}" for i in range(1, 7)]
subject_cols = ["Math", "English", "Science", "Social"]

# === FLATTEN CLASS NAMES ===
all_classes = pd.unique(df[class_cols].values.ravel())
all_classes = [c for c in all_classes if pd.notna(c) and c != ""]

# === DASH APP SETUP ===
app = dash.Dash(__name__)
app.title = "Student Class & Subject Dashboard"

app.layout = html.Div([
    html.H2("Student Performance & Class Allocation Dashboard"),
    
    html.Label("Filter by Class:"),
    dcc.Dropdown(
        id="class-filter",
        options=[{"label": c, "value": c} for c in sorted(all_classes)],
        placeholder="Select a class...",
        multi=False
    ),
    
    dcc.Graph(id="subject-heatmap"),
    
    html.H3("Class Assignment Table"),
    html.Div(id="class-table")
])

# === CALLBACK ===
@app.callback(
    Output("subject-heatmap", "figure"),
    Output("class-table", "children"),
    Input("class-filter", "value")
)
def update_dashboard(selected_class):
    if selected_class:
        mask = df[class_cols].apply(lambda row: selected_class in row.values, axis=1)
        filtered_df = df[mask].copy()
    else:
        filtered_df = df.copy()

    heatmap_df = filtered_df[["Student"] + subject_cols].set_index("Student")

    fig = px.imshow(
        heatmap_df,
        text_auto=True,
        color_continuous_scale="Blues",
        title="Subject Scores Heatmap",
        aspect="auto"
    )

    table_cols = ["Student"] + class_cols
    table_html = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in table_cols])),
        html.Tbody([
            html.Tr([html.Td(filtered_df.iloc[i][col]) for col in table_cols])
            for i in range(len(filtered_df))
        ])
    ])

    return fig, table_html

# === RUN APP ===
if __name__ == '__main__':
    app.run(debug=True)
